<a href="https://colab.research.google.com/github/jlopetegui98/NER-ClinicalTrials-Elegibility-Criteria/blob/main/Roberta%2BLLM/mistral7b-full-evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# uncomment if using colab
!pip install -q -U datasets
!pip install seqeval
!pip install -q -U evaluate
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=9c703c8a5ce23d3f4fe6939221ce557f0d2990ee50d7e9a6ddbf735dbd1aeef7
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextGenerationPipeline
import torch
import os
from utils import *

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset

In [4]:
import re

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# dict for the entities (entity to int value)
simple_ent = {"Condition", "Value", "Drug", "Procedure", "Measurement", "Temporal", "Observation", "Person", "Device"}
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Drug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16,
    "B-Device": 17,
    "I-Device": 18
}

entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [7]:
root = '..'
root = './drive/MyDrive/TER-LISN-2024'
data_path = f'{root}/data'
models_path = f'{root}/models'

In [8]:
model_name = "mistralai/Mistral-7B-v0.1"

In [9]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# import tokenizer for mistral-7B
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [11]:
pipe  = TextGenerationPipeline(model = model, tokenizer = tokenizer)

In [12]:
dataset = load_dataset('JavierLopetegui/chia_v1')

In [13]:
# for each sentence save the text
def generate_sentences_from_tokens(sentences):
    texts_sentences = []
    sentences_tokens = sentences['tokens']
    for sentence in sentences_tokens:
        sent_text = " ".join(sentence)
        texts_sentences.append(sent_text)
    sentences['text'] = texts_sentences
    return sentences

In [14]:
def build_prompts(sentences, prompt_type=2):
    sentences_prompts = []
    for sent in sentences['text']:
        prompt = build_prompt(sent, prompt_type)
        sentences_prompts.append(prompt)
    sentences['prompt'] = sentences_prompts
    return sentences

In [15]:
dataset = dataset.map(lambda x: generate_sentences_from_tokens(x), batched = True)
dataset_prompt2 = dataset.map(lambda x: build_prompts(x, prompt_type=2), batched = True)

In [16]:
test_dataset_p2 = dataset_prompt2['test']

In [17]:
# keep just the prompt column
test_dataset_p2 = test_dataset_p2.remove_columns(['tokens', 'text', 'ner_tags', 'file'])

In [ ]:
# data_loader_p2 = DataLoader(test_dataset_p2, batch_size=4, shuffle=False)

In [ ]:
# generated_sentences_p2 = pipe(batch['prompt'], max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole"))

In [18]:
from tqdm import tqdm

In [19]:
generated_sentences_p2 = []
for sentence in tqdm(test_dataset_p2['prompt']):
    sentence += '\noutput: '
    output = pipe(sentence, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text']
    output = output.split('\n')[0]
    generated_sentences_p2.append(output)

100%|██████████| 1307/1307 [14:54:25<00:00, 41.06s/it]


In [20]:
len(generated_sentences_p2)

1307

In [21]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [22]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True)

    labels = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_sentence['labels'] = labels
    return tokenized_sentence

**Standarizing true annotations**

In [23]:
def parse_ann2bio(sentence, pattern, pattern1, pattern2):
    # if sentence[-1] == "\n":
    #     sentence = sentence[:-2] # remove the \n and a final point wrongly added
    # else:
    #     sentence = sentence[:-1] # remove the final point wrongly added

    # find the entities
    occurrences = re.finditer(pattern, sentence)
    indexes = [(match.start(), match.end()) for match in occurrences]

    annotation = []
    i = 0


    # create the bio list
    for beg, end in indexes:
        if beg > i:
            annotation.extend([(word, "O") for word in sentence[i:beg].split()])
        entity = sentence[beg:end]
        entity_name = re.search(pattern1, entity).group(1)
        entity = entity.replace(f'<{entity_name}>', "").replace(f'</{entity_name}>', "")
        split_entity = entity.split()
        annotation.append((split_entity[0], "B-" + entity_name))
        annotation.extend([(word, "I-" + entity_name) for word in split_entity[1:]])
        i = end
    annotation.extend([(word, "O") for word in sentence[i:].split()])

    # check punctuation sign in tokens and put them as individual tokens
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    new_annotation = []
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))

        else:
            new_annotation.append((word, tag))


    return new_annotation

In [24]:
new_true_annotations = []
for sent in dataset['test']:
    annotation = []
    for word, tag in zip(sent['tokens'], sent['ner_tags']):
        annotation.append((word, entities_list[tag]))
    new_annotation = []
    ps = r'(\.|\,|\:|\;|\!|\?|\-|\(|\)|\[|\]|\{|\}|\")'
    for i,(word, tag) in enumerate(annotation):
        if re.search(ps, word):
            # find the ocurrences of the punctuation signs
            occurrences = re.finditer(ps, word)
            indexes = [(match.start(), match.end()) for match in occurrences]
            # create the new tokens
            last = 0
            for j, (beg, end) in enumerate(indexes):
                if beg > last:
                    new_annotation.append((word[last:beg], tag))
                if tag != "O":
                    label = f'I-{tag.split("-")[1]}'
                else:
                    label = "O"
                if end < len(word) or (i < len(annotation) - 1 and annotation[i+1][1] == label):
                    new_annotation.append((word[beg:end], label))
                else:
                    new_annotation.append((word[beg:end], 'O'))
                last = end
            if last < len(word):
                new_annotation.append((word[last:], label))
        else:
            new_annotation.append((word, tag))
    new_true_annotations.append(new_annotation)
len(new_true_annotations)

1307

In [25]:
true_annotations = []
for sent in new_true_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    true_annotations.append(dicc_sent)
len(true_annotations)

1307

In [26]:
true_df = pd.DataFrame(true_annotations)
true_ann_dataset = Dataset.from_pandas(true_df)

In [27]:
true_ann_dataset = true_ann_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

In [28]:
def get_labels(p):
    predictions, labels = p
    # Remove ignored index (special tokens)
    predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return predictions, labels

In [29]:
from eval_file import *

**Evaluating prompt 2**

In [30]:
pattern1 = r'<(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern2 = r'</(Person|Condition|Value|Drug|Procedure|Measurement|Temporal|Observation|Device)>'
pattern = f'{pattern1}.*?{pattern2}'

In [31]:
new_p2_annotations = []
for sent in generated_sentences_p2:
    annotation = parse_ann2bio(sent, pattern, pattern1, pattern2)
    new_p2_annotations.append(annotation)
len(new_p2_annotations)

1307

In [32]:
p2_annotations = []
for sent in new_p2_annotations:
    dicc_sent = {"tokens":[], "ner_tags":[]}
    for word, tag in sent:
        dicc_sent["tokens"].append(word)
        dicc_sent["ner_tags"].append(sel_ent[tag])
    p2_annotations.append(dicc_sent)
len(p2_annotations)

1307

In [33]:
p2_df = pd.DataFrame(p2_annotations)
p2_dataset = Dataset.from_pandas(p2_df)

In [34]:
p2_dataset = p2_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

In [35]:
# keep just sentences with the same length
sentences_to_evaluate_p2 = []
sentences_to_evaluate_true = []

for i in range(len(p2_dataset)):
    if len(p2_dataset['labels'][i]) == len(true_ann_dataset['labels'][i]):
        sentences_to_evaluate_p2.append(p2_dataset['labels'][i])
        sentences_to_evaluate_true.append(true_ann_dataset['labels'][i])

print(len(sentences_to_evaluate_p2)/len(p2_dataset))

0.8332058148431523


In [36]:
evaluator = BioEval()

In [37]:
pred_labels, true_labels = get_labels((sentences_to_evaluate_p2, sentences_to_evaluate_true))

In [38]:
evaluator.evaluate_annotations(true_labels, pred_labels, do_lower=True)

In [39]:
evaluator.performance

{'overall': {'acc': 0.6011,
  'strict': {'precision': 0.5346146303196705,
   'recall': 0.512406015037594,
   'f_score': 0.5232747864478356},
  'relax': {'precision': 0.6556187487742695,
   'recall': 0.6283834586466165,
   'f_score': 0.6417122564545542}},
 'category': {'strict': {'drug': {'precision': 0.5301204819277109,
    'recall': 0.5759162303664922,
    'f_score': 0.5520702634880803},
   'condition': {'precision': 0.6060528340600154,
    'recall': 0.6465116279069767,
    'f_score': 0.6256288059306326},
   'measurement': {'precision': 0.12238805970149254,
    'recall': 0.1607843137254902,
    'f_score': 0.13898305084745763},
   'procedure': {'precision': 0.3983050847457627,
    'recall': 0.3745019920318725,
    'f_score': 0.38603696098562634},
   'observation': {'precision': 0.030612244897959183,
    'recall': 0.024,
    'f_score': 0.026905829596412554},
   'person': {'precision': 0.02857142857142857,
    'recall': 0.02247191011235955,
    'f_score': 0.025157232704402517},
   'tempo

In [40]:
evaluator.save_evaluation('eval_p2_90.json')

In [41]:
evaluator.get_counts()

{'expect': {'overall': 5320,
  'drug': 382,
  'condition': 3655,
  'measurement': 255,
  'value': 301,
  'procedure': 251,
  'temporal': 242,
  'observation': 125,
  'person': 89,
  'device': 20},
 'prediction': {'strict': {'overall': {'total': 5099,
    'true': 2726,
    'false': 2373},
   'drug': {'total': 415, 'true': 220, 'false': 195},
   'condition': {'total': 3899, 'true': 2363, 'false': 1536},
   'measurement': {'total': 335, 'true': 41, 'false': 294},
   'procedure': {'total': 236, 'true': 94, 'false': 142},
   'observation': {'total': 98, 'true': 3, 'false': 95},
   'person': {'total': 70, 'true': 2, 'false': 68},
   'temporal': {'total': 23, 'true': 0, 'false': 23},
   'value': {'total': 22, 'true': 3, 'false': 19},
   'device': {'total': 1, 'true': 0, 'false': 1}},
  'relax': {'overall': {'total': 5099, 'true': 3343, 'false': 1756},
   'drug': {'total': 415, 'true': 265, 'false': 150},
   'condition': {'total': 3899, 'true': 2832, 'false': 1067},
   'measurement': {'total':